In [1]:
import json
import sys
import os
import datetime
from finpricing.model import BondCurveAnalyticsHelper, BondCurveSolver
from finpricing.market import SurvivalCurveStep, SurvivalCurveNelsonSiegel
# for docs folder to lookf or tests
sys.path.append(r'../tests')
from testing_utils.read_data import get_sample_bonds_portfolio, get_sample_discount_curve

In [2]:
bonds, dirty_prices = get_sample_bonds_portfolio()
discount_curve = get_sample_discount_curve()
valuation_date = datetime.date(2023, 10, 9)
settlement_date = datetime.date(2023, 10, 11)

In [3]:
for bond, dirty_price in zip(bonds, dirty_prices):
    print(bond.maturity_date, dirty_price)

Fri (2024, 03, 01) 100.62719107573301
Sat (2025, 02, 15) 102.85712222222223
Sun (2025, 06, 15) 97.45796188119483
Sun (2025, 06, 15) 97.45796188119483
Sun (2026, 11, 15) 97.87555555555555
Sat (2028, 04, 15) 98.73763333333333
Wed (2028, 11, 15) 102.61128888888888
Tue (2029, 05, 15) 104.84126718711933
Thu (2029, 11, 15) 98.47593444444445
Sat (2030, 06, 15) 95.09173333333334


In [4]:
analyticsHelper = BondCurveAnalyticsHelper(bonds)
analyticsHelper.discount_curves = discount_curve
analyticsHelper.recovery_rates = 0.
analyticsHelper.settlement_dates = settlement_date
analyticsHelper.survival_curves = SurvivalCurveStep(anchor_date=valuation_date,
                                                    dates = [valuation_date],
                                                    hazard_rates=[0.],)
analyticsHelper.get_bond_bases(
    valuation_date=valuation_date,
    dirty_prices=dirty_prices,
)

[0.0020111573135329487,
 0.005341494775240267,
 0.007553695931881768,
 0.007553695931881768,
 0.008962251418909613,
 0.010159900495730808,
 0.01030307821763447,
 0.010973208439666463,
 0.012124085181598864,
 0.012556560524848905]

In [5]:
analyticsHelper.recovery_rates = 0.4
analyticsHelper.survival_curves = SurvivalCurveNelsonSiegel(anchor_date=valuation_date,
                                                            pivot_dates=[datetime.date(2025, 6, 15),
                                                                         datetime.date(2029, 5, 15)])
analyticsHelper.valuation_date = valuation_date

bondCurveSolver = BondCurveSolver(
    bondAnalyticsHelper = analyticsHelper
)

In [6]:
res = bondCurveSolver.solve(
    dirty_prices=dirty_prices,
)

In [7]:
res

(array([ 9.22536675e-05, -6.83825155e-02,  5.67154797e-02]), 2)